In [8]:
import pandas as pd
import tensorflow as tf
from glob import glob
import numpy as np
from easydict import EasyDict as edict
import os
import time
import sys

sys.path.insert(0,os.path.expanduser('~/git/torchseg'))
print(sys.path)

from utils.config import load_config
from utils.summary_to_csv import config_to_log,load_config,load_log,edict_to_pandas

def summary(rootpath,tags,filter_str=None,):
    config_files=glob(os.path.join(rootpath,'**','config.txt'),recursive=True)
    if filter_str is not None:
        config_files=[f for f in config_files if f.find(filter_str)>=0]
    
    tasks=pd.DataFrame()
    for cfg in config_files:
        log=config_to_log(cfg)
        if log is not None:
            ed=load_config(cfg)
            metrics=load_log(log,tags)
            for key,value in metrics.items():
                ed[key]=value
                
            task=edict_to_pandas(ed)
            tasks=tasks.append(task,ignore_index=True,sort=False)
        else:
            print('cannot find log file for',cfg)
            
    return tasks

['/home/yzbx/git/torchseg', '/home/yzbx/git/torchseg', '', '/home/yzbx/git/torchseg/notebooks', '/home/yzbx/git/gnu/models/research', '/home/yzbx/git/gnu/models/research/slim', '/home/yzbx/bin/miniconda3/envs/new/lib/python36.zip', '/home/yzbx/bin/miniconda3/envs/new/lib/python3.6', '/home/yzbx/bin/miniconda3/envs/new/lib/python3.6/lib-dynload', '/home/yzbx/bin/miniconda3/envs/new/lib/python3.6/site-packages', '/home/yzbx/bin/miniconda3/envs/new/lib/python3.6/site-packages/cityscapesscripts-1.0.0-py3.6-linux-x86_64.egg', '/home/yzbx/bin/miniconda3/envs/new/lib/python3.6/site-packages/torchvision-0.2.1-py3.6.egg', '/home/yzbx/bin/miniconda3/envs/new/lib/python3.6/site-packages/IPython/extensions', '/home/yzbx/.ipython']


In [11]:
rootpath=os.path.expanduser('~/tmp/logs/pytorch')
tags=['train/iou','val/iou']
filters=['caffe','pytorch','cityscapes','-1,1','0,1']

group_str='note'
group_tags=tags.copy()
group_tags.append(group_str)
    
tasks=summary(rootpath,tags)
filter_tasks=pd.DataFrame()
for filter_str in filters:
    filter_tasks=filter_tasks.append(tasks[tasks[group_str]==filter_str][group_tags],ignore_index=True,sort=False)

print(filter_tasks.to_string())

   train/iou   val/iou        note
0   0.459440  0.403973       caffe
1   0.460660  0.405963     pytorch
2   0.463188  0.416964  cityscapes
3   0.460112  0.400258        -1,1
4   0.460654  0.401508         0,1


In [3]:
rootpath=os.path.expanduser('~/tmp/logs/pytorch')
tags=['train/iou','val/iou']
configs=[
    {'filter_str':'hyperopt004',
    'group_str':'norm_ways'},
    {'filter_str':'hyperopt005',
    'group_str':'l2_reg'},
    {'filter_str':'hyperopt006',
    'group_str':'l2_reg'},
    {'filter_str':'hyperopt007',
    'group_str':'norm_ways'},
    {'filter_str':'hyperopt008',
    'group_str':'norm_ways'},
    {'filter_str':'hyperopt009',
    'group_str':'norm_ways'},
    {'filter_str':'hyperopt010',
    'group_str':'norm_ways'},
]

pd_strs=[]
for config in configs:
    tasks=summary(rootpath,tags,config['filter_str'])
    group_str=config['group_str']
    group_tags=tags.copy()
    group_tags.append(group_str)
    max_str=tasks[group_tags].groupby([group_str]).max().to_string()
    mean_str=tasks[group_tags].groupby([group_str]).mean().to_string()
    
    pd_strs.append((max_str,mean_str))

In [6]:
for d,(max_str,mean_str) in zip(configs,pd_strs):
    print('max',d['filter_str'],d['group_str'],'*'*30)
    print(max_str)
    print('mean',d['filter_str'],d['group_str'],'*'*30)
    print(mean_str)

max hyperopt004 norm_ways ******************************
            train/iou   val/iou
norm_ways                      
-1,1         0.737600  0.222334
0,1          0.733858  0.206601
caffe        0.737186  0.214802
cityscapes   0.743215  0.231846
pytorch      0.730023  0.208678
mean hyperopt004 norm_ways ******************************
            train/iou   val/iou
norm_ways                      
-1,1         0.731978  0.186179
0,1          0.728715  0.189355
caffe        0.711750  0.191525
cityscapes   0.727899  0.200267
pytorch      0.727013  0.187163
max hyperopt005 l2_reg ******************************
        train/iou   val/iou
l2_reg                     
0.0001   0.745121  0.221688
0.001    0.725888  0.221025
0.01     0.711914  0.220541
0.1      0.533035  0.197450
1e-05    0.738100  0.209281
mean hyperopt005 l2_reg ******************************
        train/iou   val/iou
l2_reg                     
0.0001   0.727874  0.191264
0.001    0.721769  0.185075
0.01     0.694604  0